In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from time import time, sleep
import numpy as np
import cvzone
from pynput.keyboard import Controller

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Hand detector
detector = HandDetector(detectionCon=0.85, maxHands=1)
keyboard = Controller()

# Keys layout
keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", "←"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]
finalText = ""
clickCooldown = 0.4  # seconds
lastClickTime = 0


class Button:
    def __init__(self, pos, text, size=[85, 85]):
        self.pos = pos
        self.text = text
        self.size = size


def drawAll(img, buttonList):
    imgTransparent = np.zeros_like(img, np.uint8)
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cv2.rectangle(imgTransparent, (x, y), (x + w, y + h), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgTransparent, button.text, (x + 20, y + 65),
                    cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
        cvzone.cornerRect(imgTransparent, (x, y, w, h), 20, rt=0)

    # Blend the transparent layer with main image
    return cv2.addWeighted(img, 1, imgTransparent, 0.4, 0)


# Create buttons
buttonList = []
for i in range(len(keys)):
    for j, key in enumerate(keys[i]):
        x = 100 * j + 50
        y = 100 * i + 50
        buttonList.append(Button([x, y], key))

# Main loop
while True:
    success, img = cap.read()
    if not success:
        print("Camera error")
        break

    hands, img = detector.findHands(img)
    img = drawAll(img, buttonList)

    if hands:
        lmList = hands[0]['lmList']
        cursor = lmList[8]  # Index tip

        for button in buttonList:
            x, y = button.pos
            w, h = button.size

            if x < cursor[0] < x + w and y < cursor[1] < y + h:
                cv2.rectangle(img, (x - 5, y - 5), (x + w + 5, y + h + 5), (175, 0, 175), cv2.FILLED)
                cv2.putText(img, button.text, (x + 20, y + 65),
                            cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)

                # Distance between index and middle fingertips
                l, _, _ = detector.findDistance(lmList[8][:2], lmList[12][:2], img)

                if l < 40 and (time() - lastClickTime > clickCooldown):
                    lastClickTime = time()

                    if button.text == "←":
                        finalText = finalText[:-1]  # Backspace
                    else:
                        finalText += button.text
                        keyboard.press(button.text)

    # Display the typed text box
    cv2.rectangle(img, (50, 350), (1200, 450), (175, 0, 175), cv2.FILLED)
    cv2.putText(img, finalText, (60, 430),
                cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 5)

    # Show window
    cv2.imshow("Virtual Keyboard", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
